In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import regex as re
import time
from datetime import datetime
import os




# prod
user = 'read_only'
pwd = 'WHC63fKyjudfSz2U'
host = 'ezcapc-webdb-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
db_name = 'CAPC_APIGATEWAY'
env='production'

db_connection_str = 'mysql+pymysql://{}:{}@{}/{}'.format(user,pwd,host,db_name)
db_connection = create_engine(db_connection_str)

In [ ]:
query = '''
SELECT 
    dm.account_id, orignal_path
FROM
    document_mst AS dm
        INNER JOIN
    account_mst AS am ON dm.account_id = am.id
        INNER JOIN
     document_processing_detail AS dpd ON dm.id = dpd.document_id
WHERE
    am.facility_id = 150
        AND dm.is_physician_document = 1
        AND dpd.created_date >= '2023-07-24';
'''

In [ ]:
df_query = pd.read_sql(query, con=db_connection)

In [ ]:
df_query.columns 

In [ ]:
df_query

In [ ]:
df_query

In [ ]:
name_list = df_query.account_number.tolist()

In [ ]:
name_list

In [ ]:
import os
import shutil

source_folder = r"/home/trishalaswain/Documents/analysis/data/download_scripts_and_tars/OCR_PROJECT_TEST/PDF/"
# destination_folder = r"/home/trishalaswain/Documents/analysis/data/download_scripts_and_tars/OCR_PROJECT_TEST/PDF/"

for sub1 in os.listdir(source_folder):
    # pdf_name = sub1.replace(".json", "")
    pdf_name = sub1.replace(".pdf", "")

    # print(pdf_name.tr)
    # if pdf_name[1:] not in name_list:
    if pdf_name not in name_list:
        # shutil.move(source_folder +sub1, destination_folder+sub1)
        os.remove(source_folder +sub1)

In [ ]:
df_query.columns

In [ ]:
print(df_query.groupby('facility_id').size().to_markdown())

In [ ]:
for account_number, orignal_path in zip(df_query.account_number, df_query.orignal_path):
    if orignal_path == None or account_number ==None:
        continue

# fac_name=file_path.split("/")[2]
# cli=file_path.split("/")[0]
# fac_list.append(fac_name)

# original path
    print("aws s3 cp 's3://ezdi-{}-bucket/ezcapc/{}' {}.pdf".format(env,orignal_path,account_number))